Data preprossing for column rename, drop unwanted columns and save it ias new file.

In [108]:
import os   # for interacting with the operating system,  include many functions to interact with the file system
import numpy as np  # Python library used for working with arrays
import pandas as pd  #data manipulation and analysis

In [109]:
# list all files in directory - Raw Data
os.getcwd()

'c:\\Users\\shyam.kalliadan\\Desktop\\ESR10\\programming\\ESR_10'

In [110]:
raw_data_directory = os.listdir('Raw_data')
processed_data_directory = os.listdir('processed_data')

In [111]:
def mapper(old_name):
    x = re.sub("^(sub_DAQ.mod_LogicLev.cntr)\\_([a-zA-Z0-9]+)\\.([a-zA-Z]+)\\_", "", old_name)
    x = re.sub("\\.(a_var)$", "", x)
    x = re.sub(" and ", "_", x)
    return x

In [112]:
for raw_file in raw_data_directory:
    if raw_file not in processed_data_directory:
        raw_file_path = os.path.join('Raw_data', raw_file)
        #data_preprocessing(raw_file_path)
        raw_data_frame = pd.read_csv(raw_file_path, sep=';')
        raw_data = raw_data_frame.rename(columns=mapper)
        save_path = os.path.join('processed_data', raw_file)
        raw_data.to_csv(save_path, index = False)